In [20]:
import pandas as pd
import helpers.processing_helpers as ph

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer

from sklearn.neural_network import  MLPRegressor

from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline


In [33]:
df_dev = pd.read_csv("./dataset/development.csv")

In [44]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_dev.drop(columns=ph.get_column_names(features, noise_indexes))
df = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms', 'area'], acc_idxs))

In [45]:
score = make_scorer(ph.mean_euclid_dist, greater_is_better=False)

In [46]:
# These will be the constant parameters

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', MLPRegressor(random_state=42, max_iter=200, n_iter_no_change=50, learning_rate_init=0.01))
])

In [47]:
y_train = df[['x', 'y']].copy()

X_train = df.drop(columns=['x', 'y'])

In [51]:
param_grid = {'clf__hidden_layer_sizes' : [(50,),
                                            (25, 25),
                                            (35, 15),
                                            (20, 10, 20),
                                            (25, 15, 10),
                                            (20, 15, 10, 5),
                                            (15, 10, 10, 15)],
            'activation' : ['relu', 
                              'logistic', 
                              'tanh', 
                              'identity'],
            'learning_rate_init': [0.01, 0.001],
            'learning_rate': ['constant', 'invscaling', 'adaptive'],
            'solver': ['adam', 'sgd', 'lbfgs']

              }

In [52]:
gridsearch = RandomizedSearchCV(pipe, param_grid, scoring=score, cv=3, verbose=2)
gridsearch.fit(X_train, y_train)

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 3 candidates, totalling 6 fits
Iteration 1, loss = 12476.91642696
Iteration 2, loss = 1701.87472122
Iteration 3, loss = 646.01684514
Iteration 4, loss = 324.03285687
Iteration 5, loss = 175.15264284
Iteration 6, loss = 97.32800108
Iteration 7, loss = 64.30712714
Iteration 8, loss = 77.17069816
Iteration 9, loss = 45.62285844
Iteration 10, loss = 65.01094176
Iteration 11, loss = 124.52389398
Iteration 12, loss = 32.50848970
Iteration 13, loss = 52.90667962
Iteration 14, loss = 83.55195840
Iteration 15, loss = 43.76390283
Iteration 16, loss = 83.00284752
Iteration 17, loss = 54.99950965
Iteration 18, loss = 45.13281692
Iteration 19, loss = 64.99761304
Iteration 20, loss = 59.78991050
Iteration 21, loss = 36.80326381
Iteration 22, loss = 55.08150041
Iteration 23, loss = 66.04854237
Iteration 24, loss = 35.53814980
Iteration 25, loss = 54.66363994
Iteration 26, loss = 44.62134056
Iteration 27, loss = 39.98376974
Iteration 28, loss = 57.75110244
Iteration 29, los

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 8027.86437007
Iteration 2, loss = 1467.45210186
Iteration 3, loss = 512.82774784
Iteration 4, loss = 229.00750994
Iteration 5, loss = 121.19102992
Iteration 6, loss = 62.27620708
Iteration 7, loss = 64.08477093
Iteration 8, loss = 55.79962132
Iteration 9, loss = 38.99814447
Iteration 10, loss = 48.35815299
Iteration 11, loss = 78.15915010
Iteration 12, loss = 49.36284763
Iteration 13, loss = 70.38944065
Iteration 14, loss = 40.03010970
Iteration 15, loss = 38.99511652
Iteration 16, loss = 54.57515521
Iteration 17, loss = 38.03828740
Iteration 18, loss = 47.71726854
Iteration 19, loss = 44.77633976
Iteration 20, loss = 40.58537966
Iteration 21, loss = 46.96459667
Iteration 22, loss = 39.79053485
Iteration 23, loss = 32.09421116
Iteration 24, loss = 44.14699807
Iteration 25, loss = 39.07180948
Iteration 26, loss = 34.15333840
Iteration 27, loss = 33.03196401
Iteration 28, loss = 43.00503466
Iteration 29, loss = 24.39621139
Iteration 30, loss = 33.35304160
Iteration 31

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Iteration 1, loss = 4027.14480150
Iteration 2, loss = 289.78982764
Iteration 3, loss = 77.43321550
Iteration 4, loss = 44.55176176
Iteration 5, loss = 42.04972750
Iteration 6, loss = 34.34948195
Iteration 7, loss = 33.01159778
Iteration 8, loss = 33.18061441
Iteration 9, loss = 34.89048448
Iteration 10, loss = 30.49709237
Iteration 11, loss = 48.40548871
Iteration 12, loss = 26.05190563
Iteration 13, loss = 26.11229126
Iteration 14, loss = 181.39050136
Iteration 15, loss = 42.15313121
Iteration 16, loss = 27.53808430
Iteration 17, loss = 28.85322165
Iteration 18, loss = 22.16913270
Iteration 19, loss = 20.08923011
Iteration 20, loss = 17.83796005
Iteration 21, loss = 17.61153616
Iteration 22, loss = 38.44514929
Iteration 23, loss = 50.36096301
Iteration 24, loss = 22.03884015
Iteration 25, loss = 16.83749098
Iteration 26, loss = 16.14717546
Iteration 27, loss = 16.19348134
Iteration 28, loss = 15.33871356
Iteration 29, loss = 15.30206180
Iteration 30, loss = 16.89929246
Iteration 31, l

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 6088.23951202
Iteration 2, loss = 320.24730860
Iteration 3, loss = 111.51786205
Iteration 4, loss = 87.96363576
Iteration 5, loss = 54.78344094
Iteration 6, loss = 41.60994654
Iteration 7, loss = 45.05554019
Iteration 8, loss = 39.40811087
Iteration 9, loss = 44.39127653
Iteration 10, loss = 35.89191625
Iteration 11, loss = 50.27450081
Iteration 12, loss = 33.45792945
Iteration 13, loss = 45.26990214
Iteration 14, loss = 29.12530637
Iteration 15, loss = 62.84037896
Iteration 16, loss = 32.00469521
Iteration 17, loss = 32.45966901
Iteration 18, loss = 26.88837752
Iteration 19, loss = 40.26918716
Iteration 20, loss = 29.67582756
Iteration 21, loss = 29.72316586
Iteration 22, loss = 35.65800701
Iteration 23, loss = 31.73199655
Iteration 24, loss = 38.04391344
Iteration 25, loss = 28.73908541
Iteration 26, loss = 29.19065348
Iteration 27, loss = 24.25823347
Iteration 28, loss = 30.58250790
Iteration 29, loss = 56.09230353
Iteration 30, loss = 50.44054097
Iteration 31, l

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=2,
                   estimator=Pipeline(steps=[('scale', StandardScaler()),
                                             ('clf',
                                              MLPRegressor(learning_rate_init=0.01,
                                                           max_iter=100,
                                                           n_iter_no_change=50,
                                                           random_state=42,
                                                           verbose=1))]),
                   param_distributions={'clf__hidden_layer_sizes': [(50,),
                                                                    (25, 25),
                                                                    (15, 10, 10,
                                                                     15)]},
                   scoring=make_scorer(mean_euclid_dist, greater_is_better=False, response_method='predict'),
                   verbose=2)

In [53]:
results = pd.concat([pd.DataFrame(gridsearch.cv_results_["params"]),
                     pd.DataFrame(gridsearch.cv_results_["mean_test_score"], columns=["MED"]),
                     pd.DataFrame(gridsearch.cv_results_["mean_fit_time"], columns=["Time"])],
                     axis=1)
df = results.sort_values('MED', ascending=False)

# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
 
# All dataframes hereafter reflect these changes.
display(df)

pd.reset_option('all')

,clf__hidden_layer_sizes,MED,Time
2,"(15, 10, 10, 15)",-69.671082,66.916625
0,"(50,)",-73.800299,43.318914
1,"(25, 25)",-84.586881,51.188641


C:\Users\Arist\AppData\Local\Temp\ipykernel_20084\3228783991.py:16: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')
